In [75]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from my_ml_package.classification import sigmoid
# import tensorflow as tf
# import torch

# read titanic data
titanic_df = pd.read_csv('data/titanic_cleaned_data.csv')

# define X and y
# Port of Embarkation: Q = Queenstown, S = Southampton
feature_cols = ['Pclass', 'Sex', 'Embarked_Q'] # , 'Embarked_S'
feature_idx2name = {i: col for i, col in enumerate(feature_cols)}
X = titanic_df[feature_cols]
y = titanic_df.Survived
X = X.to_numpy()
y = y.to_numpy()
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3, random_state=42)


In [76]:
Xtest.shape

(268, 3)

<!-- ## Intuitive Understanding of Neural Networks
* NN is analogous to the neural system of brains (**neural diagram** can be seen in the left of the figure below)
    + Brains rely on >100 billions neurons
    + Why is the model of brains so important for intelligence?  
        + communicate with each other via synapses. Each neuron can rececive up to 10K connections from/to other neurons; 
        + "Hundreds of different organs, [each carry out their own specific functions](https://human-memory.net/occipital-brain-lobe/)". 
        + occipital: process visual information; temporal: memory; frontal: planning; cerebellum: balance
    * Activation process: the process by which a neuron becomes active through connections with other neurons, which then stimulates (triggers the activation of) other connected neurons. 
    + How to represent the neurons, their connections (synapses) and functional organs mathematically?
      + input/intermediate output values; Weights or paramters    
      + Each layer consists of **input/intermediate/output values (vertices/nodes/neurons)** connected by **weights** (synapses)
    + How does the activation process of a neural network stimulate different functions?
    forward pass contains different activation functoins and architectures -->



<!-- <center><img src="https://i.loli.net/2021/08/12/7yaGWTIKlHNz6mD.png" width='500' ></center> -->

* What are the differences between linear regression, logistic regression and neural network?
    + Logistic regression has no intermediate layer, while NN has multiple intermediate layers, each with multiple neurons. 
    + i.e., a multivirate function with multiple inputs and one output vs a chain of multivariate functions with multiple inputs and multiple outputs -->
    
## Linear Function
* Notation: Let  $m$ = the number of examples, $n_i$ = the input dimension, $n_o$ = the output dimension
* The linear function:  map input $X$ to the output $y$ by finding the parameters $W$ (determined by loss function): 
*  $\hat{y} = W \cdot X$  where $\mathbf{X} \in \mathbb{R}^{n_i \times m}$ and $\mathbf{W} \in \mathbb{R}^{n_o \times n_i}$ and $\mathbf{\hat{Y}} \in \mathbb{R}^{n_o \times m}$
  
  
<!-- *  The form conforms to the implementation in Tensorflow： $\hat{y} = X \cdot W$  where $\mathbf{X} \in \mathbb{R}^{m \times n_i}$ and $\mathbf{W} \in \mathbb{R}^{n_i \times n_o}$ and $\mathbf{\hat{Y}} \in \mathbb{R}^{m \times n_o}$  -->
Sure! Here is the same explanation with weight values replaced as variables:

* Example of three neurons in a 2x3 linear layer where \( n_i = 2 \) and \( n_o = 3 \)
$$\text{Neuron}_1 = w_{11} x_{1} + w_{12} x_{2}$$
$$\text{Neuron}_2 = w_{21} x_{1} + w_{22} x_{2}$$
$$\text{Neuron}_3 = w_{31} x_{1} + w_{32} x_{2}$$

The output of this layer can be efficiently calculated via **matrix-vector product**:
$$
\left[\begin{array}{rr}
w_{11} & w_{12} & w_{13} \\
w_{21} & w_{22} & w_{23} 
\end{array}\right]\left[\begin{array}{l}
x_1 \\
x_2 \\
x_3
\end{array}\right]
$$

* We can efficiently calculate the values for \( m \) examples via matrix multiplication:
$$
\left[\begin{array}{rr}
w_{11} & w_{12} & w_{13} \\
w_{21} & w_{22} & w_{23} 
\end{array}\right]\left[\begin{array}{llll}
x_1^1 & x_1^2 & \ldots & x_1^m \\
x_2^1 & x_2^2 & \ldots & x_2^m \\
x_3^1 & x_3^2 & \ldots & x_3^m
\end{array}\right]
$$    
    

<!--     + If a bias term $b\in \mathbb{R}^{n_o \times 1}$ is added: $\hat{y} = W \cdot X + b$. But no need to do that since (1) $W$ do the job of transforming input or $b$ could be considered as  $w_{n+1} \dot x$ where $x=1$.
    + Although the first notation makes more sense for me to imagine it as the transformation in the high-dimension space, the second notation is commonly used for implementation. Here I change (use $\theta$ and transpose) the second notation into $\theta^{T} X$. Because $\theta$ which can avoid confusion for NLP notations and unify weights and bias.
    + Anyway, the linear node and bias nodes would not be shown in neural network diagrams since every neuron is assumed to have a linear node along with its corresponding bias. But we just use its output denoted as $z$ where $z = \theta^{T} X$ -->

<!-- * Function v.s. Equation: 
    + each equation represents a line in two dimensional space (or a plane) 
    + each function is an affine transformation (linear transformation with bias) which transform the whole plane e.g., $\text{hidden_neuron_1} = 2 w_{1}- w_{2}$

$$2 x_{1}- x_{2}=0$$
$$-x_{1} + 2x_{2}=3$$
This can be also represented in the matrix form to be solved efficiently.
$$
\left[\begin{array}{rr}
2 & -1 \\
-1 & 2
\end{array}\right]\left[\begin{array}{l}
x_1 \\
x_2 
\end{array}\right]=\left[\begin{array}{l}
0 \\
3
\end{array}\right]
$$ -->

Let's make a one-layer neural network.

In [80]:
# 3-2 activations
W1 = np.random.randn(2, Xtest.shape[1])
b1 = np.random.randn(2, 1)

z1 = np.dot(W1, Xtest.T) + b1
print(z1.shape)

(2, 268)


## Activation Functions
What's the problem to nest linear transformation?

<!-- > If each layer consisted only of an affine transformation, the entire function $f$ would also be affine -->


Add non-linearity into the neural network
* Sigmoid
    
    $$p =\frac{1}{1+e^{-x}}$$
    + Note that the sigmoid function is also used for calculating probability in the last layer for computing the loss
* tanh
    $$p=\frac{2}{1+e^{-2 x}}-1$$


* Thresholding Mechanism: Just like a biological neuron which fires only when the input signal is strong enough to cross a threshold, in a neural network, the activation function decides whether the node should get activated or not based on the input signal's strength. For example, the ReLU function activates a node only if the input is positive.

In [81]:
a1 = sigmoid(z1)

### Two Layers

In [82]:
W2 = np.random.randn(1, 2)
b2 = np.random.randn(1, 1)

z2 = np.dot(W2, a1) + b2
a2 = sigmoid(z2)
print(a2.shape)
yhat = (a2 > 0.5).astype(int).flatten()
print(yhat.shape)
accuracy = np.mean(yhat == ytest)
print(accuracy)

(1, 268)
(268,)
0.585820895522388


## Building Multi-layers Neural Network

In [83]:
def initialize_W_b(n_i, N_o):
    """ Initialize weights and biases for a neural network with n_i input neurons and N_o output neurons in each layer.
    Args:
        n_i: number of input neurons
        N_o: list of number of output neurons in each layer
    Returns:
        W: list of weight matrices, each with shape (n_o, n_i)
        b: list of bias vectors, eahc with shape (n_o, 1)
    """
    W = []
    b = []
    for n_o in N_o:
        W.append(np.random.uniform(-1/np.sqrt(n_i), 1/np.sqrt(n_i), (n_o, n_i))) # shape (n_i, n_o)
        b.append(np.random.uniform(-1, 1, n_o).reshape(-1, 1)) # shape (n_o, 1)
        n_i = n_o
    return W, b
# 3-2-1 activations
N_o = [2,  1]
W_layers, b_layers = initialize_W_b(Xtest.shape[1], N_o )
print("Number of layers: ", len(W_layers))


Number of layers:  2


## Optimizing W, b
Recap: How to optimize weights of linear+sigmoid functions in logistic regression?
  <!-- * Optimization algorithm: [Gradient Descent](https://medium.com/@sergioli/understanding-gradient-descent-in-pytorch-dca50926bce4)
  * Use gradients to update weights
  * [Survey Paper from Sebastian Ruder](https://arxiv.org/abs/1609.04747) -->
  
What's the challenge for neural network?
<!-- * The calculation of gradients
* Backward propagation -->

<!-- Combination of Layers/Architectures
* how model parameters are used by `torch.nn.Module`?
* how they **nest** different layers or operations?
* How does backward propagation work in pytorch `AutoGrad`? -->

In [84]:
N_o = [2,  1]
W_layers, b_layers = initialize_W_b(Xtest.shape[1], N_o )
print("Number of layers: ", len(W_layers))

learning_rate = 0.001
for _ in range(1000):
    grad_W_layers, grad_b_layers = calculate_grads_for_all_layers( W_layers, b_layers, Xtrain, ytrain)
   
    # gradient descent
    W_layers[-1] -= learning_rate * grad_W_layers[-1]
    b_layers[-1] -= learning_rate * grad_b_layers[-1]
    W_layers[-2] -= learning_rate * grad_W_layers[-2]
    b_layers[-2] -= learning_rate * grad_b_layers[-2]

# predict
activation = Xtest.T
for i in range(len(N_o)):
    z = np.dot(W_layers[i], activation) + b_layers[i]
    activation = sigmoid(z) # shape: (n_o, n_samples)

yhat = (activation > 0.5).astype(int).flatten()
accuracy = np.mean(yhat == ytest)
print(accuracy)

Number of layers:  2
0.7910447761194029


In [49]:
def compute_gradients_for_output_layer(last_activation, zs, y, a_prev):
    """
    Compute the gradients of the loss with respect to the output layer's activation, z, weights, and biases.
    
    1. Compute Gradient of Loss w.r.t. Output (grad_al):
       - \partial L / \partial a_l: grad_al = (activation - ytrain) / (activation * (1 - activation))
    
    2. Compute Gradient of Loss w.r.t. z of the last layer (grad_zl):
       - \partial L / \partial z_l 
         = \partial L /  \partial a_l 
                       * \partial a_l /  \partial z_l 
         = grad_al * sigmoid(zs[-1]) * (1 - sigmoid(zs[-1]))
    
    3. Compute Gradient of Loss w.r.t. Weights of the last layer (grad_Wl):
       - \partial L / \partial W_l 
         = \partial L /   \partial z_l 
                         *  \partial z_l /  \partial W_l 
         = grad_zl @ activations[-2].T
    """
    grad_al = (last_activation - y.reshape(1, -1)) / (last_activation * (1 - last_activation)) # shape: (n_o, n_samples)
    grad_zl = grad_al * sigmoid(zs[-1]) * (1 - sigmoid(zs[-1])) # shape: (n_o, n_samples)
    grad_Wl = grad_zl @ a_prev.T # shape: (n_o, n_i)
    grad_bl = grad_zl @ np.ones((last_activation.shape[1], 1)) # shape: (n_o, 1) 

    return grad_al, grad_zl, grad_Wl, grad_bl


def compute_gradients_for_linear_sigmoid(W_after, grad_z_after, z, a_prev):
    ############### Errors/delta #################
    # \partial L /  \partial a_{l-1} = \partial L /   \partial z_l
    #                                          * \partial z_l /  \partial a_{l-1}
    grad_a = W_after.T @ grad_z_after  # shape: (num_neurons, n_samples) 

    # \partial L /  \partial z_{l-1} = \partial L /   \partial a_{l-1}
    #                                          * \partial a_{l-1} /  \partial z_{l-1}
    grad_z = grad_a * (sigmoid(z) * (1 - sigmoid(z))) # shape: (num_neurons, n_samples)

    ################# Gradients for W_{l-1} and b_{l-1} #################
    # \partial L /  \partial W_{l-1} = \partial L /   \partial z_{l-1}
    #                                          *  \partial z_{l-1} /  \partial W_{l-1}
    grad_W = grad_z @ a_prev.T # shape: (num_neurons, num_neurons_prev) 
   
    # \partial L /  \partial b_{l-1} = \partial L /   \partial z_{l-1}
    #                                          *  \partial z_{l-1} /  \partial b_{l-1}

    grad_b = grad_z @ np.ones((z.shape[1], 1)) # shape: (num_neurons, 1) = (num_neurons, n_samples) * (n_samples, 1)

    return grad_a, grad_z, grad_W, grad_b


def calculate_grads_for_all_layers(W_layers, b_layers, Xtrain, ytrain):
    # init
    grads_W_layers = [np.zeros_like(W) for W in W_layers]
    grads_b_layers = [np.zeros_like(b) for b in b_layers]

    # forward pass
    zs = []
    activations = []
    activations.append(Xtrain.T)
    activation = Xtrain.T 
    for i in range(len(N_o)):
        z = np.dot(W_layers[i], activation) + b_layers[i]
        activation = sigmoid(z) # shape: (n_o, n_samples)
        zs.append(z)
        activations.append(activation)
    
    # backward pass: calculate gradients of the loss function w.r.t. W and b
    grad_al, grad_zl, grad_Wl, grad_bl = compute_gradients_for_output_layer(activations[-1], zs, ytrain, a_prev=activations[-2])
    grads_W_layers[-1] = grad_Wl
    grads_b_layers[-1] = grad_bl

    for i in range(2, len(N_o)+1):
        assert W_layers[-(i-1)].shape[0] == grad_zl.shape[0]
        grad_a_prev, grad_z_prev, grad_W_prev, grad_b_prev = compute_gradients_for_linear_sigmoid(W_layers[-(i-1)], grad_zl, zs[-i], a_prev=activations[-(i+1)] )
        grads_W_layers[-i] = grad_W_prev
        grads_b_layers[-i] = grad_b_prev
        grad_zl = grad_z_prev

    return grads_W_layers, grads_b_layers

    

In [ ]:

# # sklearn
# mlp = MLPClassifier(hidden_layer_sizes=(256,), \
#                     activation="relu", \
#                     solver='adam',\
#                     beta_1=0.9, beta_2=0.999, \
#                     learning_rate='constant', learning_rate_init=0.001, \
#                     max_iter=1000, random_state=0)
# for i in range(mlp.n_layers_)[:-1]:
#     print(i+1, '-th layer: ', mlp.coefs_[i].shape)
# mlp.fit(X_train, y_train)
# plt.plot(mlp.loss_curve_)
# plt.show()
# y_pred = mlp.predict(X_test)
# print(confusion_matrix(y_test, y_pred))
# print(classification_report(y_test, y_pred, target_names=target_names))


In PyTorch
* The attribute `requires_grad` to control whether the gradients should be calculated during the backward. Specifically, after performing operations (e.g., sin) on the input tensor during forward propagation, the `grad_fn` attribute would be added into the output tensor in order to calculate gradients afterward.

* Recording Tensor Operations: [The official introduction](https://pytorch.org/tutorials/beginner/blitz/autograd_tutorial.html#computational-graph) 
* The `Tensor` class is the finest class we can trace the parameters and their corresponding gradients during backward propagation
> Conceptually, autograd keeps a record of data (tensors) & all executed
operations (along with the resulting new tensors) in a directed acyclic
graph (DAG) consisting of
[`Function`](https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)
objects. In this DAG, leaves are the input tensors, roots are the output
tensors. By tracing this graph from roots to leaves, you can
automatically compute the gradients using the chain rule.
>
> ![image.png](attachment:5e01a2e2-7506-448a-8e32-71531c5f6f12.png)![image.png](attachment:235839b6-fb87-45b4-a999-19d08bc69415.png)
>
> In a forward pass, autograd does two things simultaneously:
> 
> - run the requested operation to compute a resulting tensor, and
> - maintain the operation’s *gradient function* in the DAG.
>
> The backward pass kicks off when ``.backward()`` is called on the DAG
root. ``autograd`` then:
> 
> - computes the gradients from each ``.grad_fn``,
> - accumulates them in the respective tensor’s ``.grad`` attribute, and
> - using the chain rule, propagates all the way to the leaf tensors.
> 
> Below is a visual representation of the DAG in our example. In the graph,
the arrows are in the direction of the forward pass. The nodes represent the backward functions
of each operation in the forward pass. The leaf nodes in blue represent our leaf tensors ``a`` and ``b``.
> 
> .. figure:: /_static/img/dag_autograd.png
> 
> <div class="alert alert-info"><h4>Note</h4><p>**DAGs are dynamic in PyTorch**
  An important thing to note is that the graph is recreated from scratch; after each
  ``.backward()`` call, autograd starts populating a new graph. This is
  exactly what allows you to use control flow statements in your model;
  you can change the shape, size and operations at every iteration if
  needed.</p></div>
> 
> Exclusion from the DAG
>
> ``torch.autograd`` tracks operations on all tensors which have their
``requires_grad`` flag set to ``True``. For tensors that don’t require
gradients, setting this attribute to ``False`` excludes it from the
gradient computation DAG.
>
> The output tensor of an operation will require gradients even if only a
single input tensor has ``requires_grad=True``.

In [ ]:
torch.from_numpy(X_batch.T)


# create an input tensor
X_batch = X_train[:2]
y_batch = y_train[:2]
X_batch = torch.from_numpy(X_batch.T).float()
y_batch = torch.from_numpy(y_batch)

W1 = np.random.uniform(-1/np.sqrt(n_i), 1/np.sqrt(n_i), (n_m, n_i))
W1 = torch.from_numpy(W1).float()
W1.requires_grad=True
A1= torch.matmul(W1, X_batch)
A1 = F.tanh(A1)

W2 = np.random.uniform(-1/np.sqrt(n_m), 1/np.sqrt(n_m), (5, n_m))
W2 =  torch.from_numpy(W2).float()
W2.requires_grad=True
A2= torch.matmul(W2, A1)
y_prob = F.softmax(A2.T)

# input tensor has no grad_fn
# print(X_batch.grad_fn)

loss = F.nll_loss(torch.log(y_prob), y_batch)

loss.backward()




# parameters
w = torch.Tensor([1.,2.,3.])
print('Original w grad:', w.grad)


# we create `x` containing two examples, each has 3-dimensional features
x1 = torch.Tensor([1., 2., 3.])
x2 = torch.Tensor([4., 5., 6.])

y = x1 * w
z = y.sum()
z.backward()
print('w grad with x1:', w.grad)

y = x2 * w
z = y.sum()
w.grad = None
z.backward()
print('w grad with x2:', w.grad)


# The gradients will be summed up in the batch dimension
x = torch.Tensor([[1., 2., 3.],
                  [4., 5., 6.]])
y = x * w
z = y.sum()
w.grad = None
z.backward()
print('mini-batch gradient of parameters with x1 and x2:', w.grad)

## More Complicated Neural Networks： convolution (CNN), recurrent (RNN), Transformer

* Real-world data often contains properties that are sequential or spatial in nature, such as text, audio, video, sensor readings, and images. These properties can be important in determining the meaning of the data, and therefore need to be taken into account when processing the data.

* Sequential properties refer to the fact that the positions of data points matter in explaining the overall meaning of the data, while spatial properties refer to the fact that the relative positions of data points matter in determining the meaning of the data. 
* Communication using language: we use a sequence of words to express our thoughts and ideas. 
    + The order in which we arrange these words is very important: "The cat sat on the mat." v.s. "The mat sat on the cat."
    + In order to accurately predict or classify language data, a neural network must be able to understand the relationships between words in a sentence and how they are ordered. Hence, sequential neural network architectures are designed to take into account the sequential nature of language data and can effectively learn the complex relationships between words in a sentence.

<!-- * (Optional) The way I think of why we need different transformations in neural net: Just like playing Lego we need more than regular pieces🔺 to build real-world things 🚉, 📱... , neural network consist of more than linear transformation to model the real-world problem. This depends on prior knowledge of what problem you want to solve.  For example,  
if you build ⏰, you need irregularly shaped pieces like wheel🏽⚙️. 
If you predict the increase of human population in China next year, you may use exponential function since you know  it will increase exponentially by time. 
If you predict image class, you will use convolution transformation since yoou know image has to be understood by its combination of pixels rather than each pixel separately. -->

<p style="color:red">I comment some codes for implementing different architectures here.</p>
<!-- ### 3.1 RNN

More details see [RNN/LSTM](https://www.kaggle.com/sergioli212/rnn-lstm-from-scratch/)

Code:
# input data with middle dimention 10 represents sequence length
X = torch.Tensor(np.random.randn(10, 3, 4))
n_x, m, T_x = X.shape # 3, 10, 4
n_y, n_a = 2, 5 

# pytorch linear API + tanh activation function
linear_xa = torch.nn.Linear(n_x, n_a)
linear_aa = torch.nn.Linear(n_a, n_a)
a_prev = torch.randn(m, n_a)
for t in range(T_x):
    input1 = linear_xa(X[:,:,t])
    input2 = linear_aa(a_prev)
    a_prev = torch.tanh(input1+input2)
# print(a_prev.shape)


### 3.2 CNN

# code
# keras
# tf.keras.layers.Conv1D(
#     filters,
#     kernel_size,
#     strides=1,
#     padding="valid",
#     data_format="channels_last",
#     dilation_rate=1,
#     groups=1,
#     activation=None,
#     use_bias=True,
#     kernel_initializer="glorot_uniform",
#     bias_initializer="zeros",
#     kernel_regularizer=None,
#     bias_regularizer=None,
#     activity_regularizer=None,
#     kernel_constraint=None,
#     bias_constraint=None,
#     **kwargs
# )
# The inputs are 128-length vectors with 10 timesteps, and the batch size is 4.  
input_shape = (4, 10, 128)
x = tf.random.normal(input_shape)
y = tf.keras.layers.Conv1D(32, 3, activation='relu',input_shape=input_shape[1:])(x)
print(y.shape)

# pytorch: https://pytorch.org/docs/stable/generated/torch.nn.Conv1d.html#torch.nn.Conv1d
# torch.nn.Conv1d(
#     in_channels,  # unlike keras, pytorch cannot infer the input channels since it has no placeholder for input
                    # put sequence length here
#     out_channels, 
#     kernel_size, 
#     stride=1,
#     padding=0, 
#     dilation=1, 
#     groups=1, 
#     bias=True, 
#     padding_mode='zeros')
x = tf.constant(
[[[1.,2.,3.],
  [2.,2.,1.],
  [3.,2.,6.]],

 [[4.,4.,5.],
  [5.,5.,5.],
  [6.,6.,7.,],],

 [[7.,8.,8.,],
  [8.,6.,7.,],
  [9.,8.,8.,]]] , dtype='float32')
print(x)

# keras API
max_pool_1d = tf.keras.layers.GlobalMaxPooling1D()
# max_pool_1d = tf.keras.layers.MaxPooling1D(3)
print(max_pool_1d(x))

# jax API
from jax import random
import jax.numpy as jnp
key = random.PRNGKey(1701)
x = jnp.linspace(0, 10, 500)
y = jnp.sin(x) + 0.2 * random.normal(key, shape=(500,))
window = jnp.ones(10) / 10
y_smooth = jnp.convolve(y, window, mode='same')
plt.plot(x, y, 'lightgray')
plt.plot(x, y_smooth, 'black'); -->


## 4. How to Acquire A Set of Weights Optimized for the Data/Task?
1. Randomly initialized weights
2. Define loss function, which can be used as an minimization objective function to optimize $W$.
3. Iteratively update weights to minimize the loss function using gradient descent

### 4.1 [Loss function](https://www.kaggle.com/sergioli212/cross-entropy-loss-details).
* Intuitively, it is used to quantify, given empirical data (i.e., training samples), how wrong the current model predicts in contrast to the true values of samples. The quantity is called the loss between model outputs and grouth truth.
* Let's design loss function for classification and regression problems?
  
* The formal definition relates to **Bayes Theorem** for machine learning in Week 2 and **Minimizing Negative Log Likelhood** in Week 6
    + Bayes Theorem: maximize Pr(parameters|data) $\propto$ Pr(data|parameters) * Pr(parameters)
    + Likelihood: $ L(\theta) =  Pr((X, y)|\theta) $
    + Given a set of $\theta$, the likelihood can be calculated by "the product of the probabilities for each data point". The likelihood measures whether a given $\theta$ is a good estimation. 
$ L(\theta) =  Pr((X, y)|\theta)= \prod_{i=1}^{N} Pr\left(y^{(i)}, x^{(i)} | \theta\right) =Pr\left(y^{(i)} | x^{(i)} , \theta\right)$
    + Why does it call likelihood not probability? 
    <!-- ⚠️ although we fix $\theta$ to caluculate the probability $Pr(X, y | \theta)$, we never know the only true $\theta$ to calculate the true 👉$Pr(X, y | \theta)$. The probability is really just fake. That's why we give it a new name --- `LIKELIHOOD`.  -->
    + The optimal $\theta$ should give the maximum liklihood or the minimum NLL $\rightarrow$ We try our best to find a set of $\theta$ which leads to NLL as low as possible, i.e., $ \theta = argmax L(\theta)=Pr(y | X , \theta)$. 

<!--     We normally use Negative Log Likelhood (NLL). -->

MLE for Binary Classification

* The last layer normaly has one neuron/probability from the sigmoid function, denoted as
 $ p=Pr(y = 1|x; \theta)  $
 $$\begin{aligned} NLL = -\log L(\theta) \\ &=-\sum_{i=1}^{m} y^{(i)} \log p+(1-y^{(i)}) \log (1-p)) \end{aligned}$$

* It can be considered as **KL-devergence**, which measures the similarity between two distributions. [This post](https://jaketae.github.io/study/kl-mle/) proves that "minimizing the KL divergence amounts to finding the maximum likelihood estimate of $\theta$". During the supervised learning，the ground-truth distribution is know. Hence, minizing **cross entropy** equals to the minimization of KL-devergence
$$D_{\mathrm{KL}}(Y \| \hat{Y})=-\sum_{i} y^{(\mathrm{i})} \log \frac{\hat{y}^{(\mathrm{i})}}{y^{(\mathrm{i})}}$$ 

In [ ]:
y_true = np.array([1, 0, 1])
y_prod = np.array([0.9, 0.1, 0.8])

# calculate nll
log_probs = np.multiply(y_true ,np.log(y_prod)) + np.multiply((1-y_true), np.log(1-y_prod))
cost = (-1/len(y_true)) * np.sum(log_probs)
print(cost)

* The last layer normaly has one neuron/probability from the sigmoid function, denoted as
 $ p=Pr(y = 1|x; \theta) $
 $$\begin{aligned} NLL = -\log L(\theta) \\ &=-\sum_{i=1}^{m} y^{(i)} \log p+(1-y^{(i)}) \log (1-p)) \end{aligned}$$

* It can be considered as **KL-devergence**, which measures the similarity between two distributions. [This post](https://jaketae.github.io/study/kl-mle/) proves that "minimizing the KL divergence amounts to finding the maximum likelihood estimate of $\theta$". During the supervised learning，the ground-truth distribution is know. Hence, minizing **cross entropy** equals to the minimization of KL-devergence
$$D_{\mathrm{KL}}(Y \| \hat{Y})=-\sum_{i} y^{(\mathrm{i})} \log \frac{\hat{y}^{(\mathrm{i})}}{y^{(\mathrm{i})}}$$ 

MLE for Multiclass Classification

* Softmax Funtion: Except for sigmoid function, softmax function is used to calculate the output probability distribution for more than two classes
    $$p_i =\frac{e^x_i}{\sum_{i=1}^{K} e^{x_i}} $$

In [4]:
y_prob = None
y_true = None

* How can we get y_prob? What should we change to the above neural net? 
<!-- Softmax Funtion: Except for sigmoid function, softmax function is used to calculate the output probability distribution for more than two classes
    $$p_i =\frac{e^x_i}{\sum_{i=1}^{K} e^{x_i}} $$ -->
<!-- def softmax(z):
    e_z = np.exp(z)
    return e_z / np.sum(e_z, axis=-1).reshape(-1, 1)

logits = [[4.0, 2.0, 1.0], 
          [0.0, 5.0, 1.0]]

y_prob = softmax(logits)
y_prob -->

In [ ]:
# get y_prob from the neural network


# calculate nll

In [6]:
def softmax(z):
    e_z = np.exp(z)
    return e_z / np.sum(e_z, axis=-1).reshape(-1, 1)

logits = [[4.0, 2.0, 1.0], 
          [0.0, 5.0, 1.0]]


y_prob = softmax(logits)
y_prob


# one-hot representations
labels = np.array([[1],[2]])

# L(y_pred, y) = -sum(y * log y_pred for each class) = -log y_pred[true_class_idx]
total_loss = -np.log(np.take_along_axis(y_prob, labels, axis=-1))
np.mean(total_loss)

# pytorch API
loss = F.nll_loss(torch.from_numpy(np.log(y_prob)), labels))
print(loss)
# loss = F.cross_entropy(torch.tensor(logits), torch.tensor([1, 2]))

# Tensorflow API
# tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)


## Closing Words 
* A neural network is an universal function approximator. For example, if you use softmax or sigmoid, a neural network represents conditional probability distribution. For understanding this, see [this notebook](https://www.kaggle.com/sergioli212/loss-function-and-optimization).
* the above models/functions are not trained so they are not a model fitting into any data. For understanding this, see [this notebook](https://www.kaggle.com/sergioli212/loss-function-and-optimization).
* Bullshit: The reason I would like to decompose NN architecture and modelling process is because, I think, I always see a lot of implementation packages and tutorials tend to mix up concepts or algorithms as a black box which hurts the flexibility to identify  and adjust finer pieces for solving my problem. For example, to generate adversarial text of NLP, I had a trouble to understand why we could update input instead of model parameters and what is problem of discrete text space and how to add constraints for ensuring fluency fo generating text via loss function (or adversarial objective functions here). All in all, each functions (linear, activation, stacking function, loss functions) during forward pass and their differentiation properties during backward pass should be well understood for deep learning practitioners.

## Reference
* [How to initialize weights?](https://stackoverflow.com/questions/49433936/how-to-initialize-weights-in-pytorch)